# Homework 1
## Matt Gallagher

In [2]:
from pulp import *

## Question 2

Formulate and solve the “Two Products – Three Machine” problem from Class #1 as a linear program.  Also, how would you set up this model so you could scale it?  That is, how would you structure it if you had a lot of machines and a lot of products?

In [4]:
#Create a list of all the products
Products = ["A","B"]

#Create a dictionary of the prices for products
Prices = {"A": 470,
         "B":420}

#Create a list of all the raw materials
RawMaterials = ["M1","M2","M3"]

#Create a Dictionary of Amount of each Raw Material Available
RawAvailability = {"M1": 336,
                  "M2": 336,
                  "M3": 336}

#Create a list for amount used of each raw material for each product
AmountUsed = {"A": {"M1": 2, "M2": 0, "M3": 2},
              "B":{"M1": 0, "M2": 2, "M3": 1.5}
             }

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Three Machines",LpMaximize)

# Create the Variables
product_vars = LpVariable.dicts("Prods",Products,lowBound=0,upBound=None,cat=LpContinuous)

# The objective function is added to 'prob' first
prob += lpSum([Prices[i]*product_vars[i] for i in Products]), "Total Revenue of Production Plan"

# We can enter the constraints that relate to limited amount of material
for r in RawMaterials:
    prob += lpSum([product_vars[i]*AmountUsed[i][r] for i in Products]) <= RawAvailability[r] , r

# Additional constraints
prob += product_vars["A"] >= 75
prob += product_vars["A"] <= 140
prob += product_vars["B"] <= 140

# The problem data is written to an .lp file
prob.writeLP("3Mach.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("Total Revenue of Plan = ", value(prob.objective))

for constraint in prob.constraints:
        # print(prob.constraints[constraint].name, prob.constraints[constraint].value() - prob.constraints[constraint].constant)
        print(prob.constraints[constraint].name, prob.constraints[constraint].value())
        print(prob.constraints[constraint].name, prob.constraints[constraint].constant)

Status: Optimal
Prods_A = 75.0
Prods_B = 124.0
Total Revenue of Plan =  87330.0
M1 -186.0
M1 -336
M2 -88.0
M2 -336
M3 0.0
M3 -336.0
None 0.0
None -75
None -65.0
None -140
None -16.0
None -140


## Question 3

Your non-vegetarian friend wants to determine how many units of different kinds of meat he should eat to get the right amounts of vitamins A, C, B1, and B2 over the week.  He also wants to minimize his cost.  What should his diet be?  How would you modify the linear program to make it “better?”  You can find the cost for each unit of food and the percent of daily requirements for each vitamin type in the spreadsheet. 

In [5]:
#Create a list of all the products
Meats = ["Beef","Chicken","Fish","Ham","MeatLoaf","Turkey"]

#Create a dictionary of the prices for products
Prices = {"Beef": 4.16,
         "Chicken":2.75,
         "Fish": 3.28,
         "Ham": 2.91,
         "MeatLoaf": 2.25,
         "Turkey": 2.45}

#Create a list of all the raw materials
Vitamins = ["A","C","B1","B2"]

#Create a Dictionary of Amount of each Raw Material Available
# RawAvailability = {"M1": 336,
#                   "M2": 336,
#                   "M3": 336}

#Create a list for amount used of each raw material for each product
AmountSupplied = {"Beef": {"A": .6, "C": .2, "B1": .1, "B2": .15},
              "Chicken":{"A": .08, "C": 0, "B1": .2, "B2": .2},
              "Fish": {"A": .08, "C": .1, "B1": .15, "B2": .1},
              "Ham": {"A": .4, "C": .4, "B1": .35, "B2": .1},
              "MeatLoaf": {"A": .7, "C": .3, "B1": .15, "B2": .15},
              "Turkey": {"A": .6, "C": .2, "B1": .15, "B2": .1}
             }

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Meats",LpMinimize)

# Create the Variables
product_vars = LpVariable.dicts("Meats",Meats,lowBound=0,upBound=None,cat=LpContinuous)

# The objective function is added to 'prob' first
prob += lpSum([Prices[i]*product_vars[i] for i in Meats]), "Total Cost of Meat Plan"

# We can enter the constraints that relate to limited amount of material
for r in Vitamins:
    prob += lpSum([product_vars[i]*AmountSupplied[i][r] for i in Meats]) >= 1 , r


# The problem data is written to an .lp file
prob.writeLP("Meats.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("Total Cost of Plan = ", value(prob.objective))

for constraint in prob.constraints:
        # print(prob.constraints[constraint].name, prob.constraints[constraint].value() - prob.constraints[constraint].constant)
        print(prob.constraints[constraint].name, prob.constraints[constraint].value())
        print(prob.constraints[constraint].name, prob.constraints[constraint].constant)

Status: Optimal
Meats_Beef = 0.0
Meats_Chicken = 2.5
Meats_Fish = 0.0
Meats_Ham = 0.0
Meats_MeatLoaf = 3.3333333
Meats_Turkey = 0.0
Total Cost of Plan =  14.374999925000001
A 1.53333331
A -1.0
C -1.0000000050247593e-08
C -1.0
B1 -5.000000025123796e-09
B1 -1.0
B2 -5.000000025123796e-09
B2 -1.0


## Question 4
Solve the gas blending problem introduced in class. That is, determine how much of each type of gas you should buy and what you should use it for.  Make sure you can explain the full model and especially the blending constraints.

## Question 5
Solve the nurse scheduling problem introduced in class.  You want to know the minimum number of nurses needed to cover the 24-hour period.  You can assume that the next day is the same as this one.

In [39]:
#Create a list of all the shifts
Slots = ["a12-4","a4-8","a8-12","p12-4","p4-8","p8-12"]

#Create a dictionary of required staffing for each slot
Staff_reqs = {"a12-4": 4,
         "a4-8":8,
         "a8-12": 10,
         "p12-4": 7,
         "p4-8": 12,
         "p8-12": 4}

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Nurses",LpMinimize)

# Create the Variables , aka nursing slots
product_vars = LpVariable.dicts("Slots",Slots,lowBound=0,upBound=None,cat=LpContinuous)

# The objective function is added to 'prob' first
prob += lpSum([product_vars[i] for i in Slots]), "Total Staffing of Nursing Slots"

# Constraints
# Each slot requirement must be filled by those starting that shift, and the shift before
i=0
while i < len(Slots)-1:
    prob += lpSum(product_vars[Slots[i]] + product_vars[Slots[i+1]]) >= Staff_reqs[Slots[i+1]] , Slots[i]
    i=i+1

# The first slot must be adequatelly filled
prob += product_vars[Slots[0]] >= 4 , "Start must be 4"

# The problem data is written to an .lp file
prob.writeLP("Nurses.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("Total Nurses Used = ", value(prob.objective))

for constraint in prob.constraints:
        # print(prob.constraints[constraint].name, prob.constraints[constraint].value() - prob.constraints[constraint].constant)
        print(prob.constraints[constraint].name, prob.constraints[constraint].value())
        print(prob.constraints[constraint].name, prob.constraints[constraint].constant)

Status: Optimal
Slots_a12_4 = 4.0
Slots_a4_8 = 10.0
Slots_a8_12 = 0.0
Slots_p12_4 = 8.0
Slots_p4_8 = 4.0
Slots_p8_12 = 0.0
Total Nurses Used =  26.0
a12_4 6.0
a12_4 -8
a4_8 0.0
a4_8 -10
a8_12 1.0
a8_12 -7
p12_4 0.0
p12_4 -12
p4_8 0.0
p4_8 -4
Start_must_be_4 0.0
Start_must_be_4 -4
